In [1]:
"""This file generates the mathematically optimal timetable for Lalaland School.
Precisely, we do so using Coin-or branch and cut (CBC), a Mixed Integer Program-
ming (MIP) solver retrieved from the open-source software suite Google OR-Tools. 
The script demonstrates that the mathematical model finds exactly the optimal 
timetable which scores the theoretical maximum of 124 points."""


# Import libraries
import time
import pandas as pd

# Import the Google OR-Tools linear solver wrapper. This is an interface for 
# multiple third-party Mixed Integer Programming (MIP) solvers. Find the full
# documentation at: https://developers.google.com/optimization/mip/integer_opt
from ortools.linear_solver import pywraplp


#________________________________________________________________________________________
# STEP 1 - READ IN DATA
# Read in .xlsx file of Lalaland data
AllData = pd.ExcelFile('lalaland_students_course_preferences.xlsx')
# Read in each of the four sheets in the .xlsx Lalaland file
StudentCourseData = pd.read_excel(AllData, 'StudentCourse')
TeacherCourseData = pd.read_excel(AllData, 'TeacherCourse')
TeacherBlockData = pd.read_excel(AllData, 'TeacherBlock')
CourseBlockData = pd.read_excel(AllData, 'CourseBlock')

print(StudentCourseData)
print()
print(TeacherCourseData)
print()
print(TeacherBlockData)
print()
print(CourseBlockData)


#________________________________________________________________________________________
# STEP 2 - FORMAT DATA
# Count the number of variables for Lalaland timetabling problem
# i.e. count the number of students, teachers, courses, and blocks.
num_students = StudentCourseData.shape[0]
num_teachers = TeacherCourseData.shape[0]
num_courses = TeacherCourseData.shape[1]-1
num_blocks = TeacherBlockData.shape[1]-1
# Generate four ranges, one for each variable
all_students = range(num_students)
all_teachers = range(num_teachers)
all_courses = range(num_courses)
all_blocks = range(num_blocks)
# Generate four lists, one for each variable
# (in this case the lists are pandas series objects).
StudentList=StudentCourseData["Student"]
TeacherList=TeacherCourseData["Teacher"]
CourseList=CourseBlockData["Course"]
BlockList=list(CourseBlockData)[1:5]


# Next, generate four lists of lists and call them 'StudentCourseMatrix', 
# 'TeacherCourseMatrix', 'TeacherBlockMatrix', and 'CourseBlockMatrix', respectively. 
# Our goal is to build the optimal timetable around teachers' and students' preferences. 
# Therefore, let us assign preference scores to all courses and blocks, so that, these 
# lists of lists become the input we feed into our MIP solver. 

# Generate an 8x12 matrix for students' course preferences.
# Rows are students and columns are courses.
# For every cell in the matrix assign a score value of 2 if student m 
# wishes to take course n, and let that score be 5 otherwise.
StudentCourseMatrix = [[-5 for c in all_courses] for s in all_students]
for c in range(1,num_courses+1):
    for s in all_students:
        if StudentCourseData[list(StudentCourseData)[c]][s]=='Y':
            StudentCourseMatrix[s][c-1]=2
            
             
# Generate a 6x12 matrix for teachers' course preferences.
# Rows are teachers and columns are courses.
# For every cell in the matrix assign a score value of 5 if teacher m 
# is qualified to teach course n, and let that score be -100 otherwise.
TeacherCourseMatrix = [[-100 for c in all_courses] for t in all_teachers]
for c in range(1,num_courses+1):
    for t in all_teachers:
        if TeacherCourseData[list(TeacherCourseData)[c]][t]=='Y':
            TeacherCourseMatrix[t][c-1]=5
            

# Generate a 6x4 matrix for teachers' block compatibility.
# Rows are teachers and columns are blocks.
# For every cell in the matrix assign a score value of 0 if teacher m 
# is willing to teach in block n, and let that score be -1 otherwise.
TeacherBlockMatrix = [[0 for b in all_blocks] for t in all_teachers]
for b in range(1,num_blocks+1):
    for t in all_teachers:
        if TeacherBlockData[list(TeacherBlockData)[b]][t]=='N':
            TeacherBlockMatrix[t][b-1]=-1
            

# Generate a 12x4 matrix for course-block compatibility.
# Rows are courses and columns are blocks.
# For every cell in the matrix assign a score value of 0 if course m 
# can be scheduled in block n, and let that score be -1 otherwise.
CourseBlockMatrix = [[0 for b in all_blocks] for c in all_courses]
for b in range(1,num_blocks+1):
    for c in all_courses:
        if CourseBlockData[list(CourseBlockData)[b]][c]=='N':
            CourseBlockMatrix[c][b-1]=-1

            
#________________________________________________________________________________________            
# STEP 3 - DEFINE THE MATHEMATICAL MODEL, STATE ITS CONSTRAINTS, CALL THE SOLVER
# Declare the MIP solver we use. Specifically, let us solve Lalaland timetabling problem
# using a Mixed Integer Programming (MIP) solver called Coin-or branch and cut (CBC).
# Now create the MIP solver with the CBC backend.
solver = pywraplp.Solver('St. Margarets School', 
                         pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

# Start the timer to compute how long it takes for the solver to output the optimal solution
start_time = time.time()

# Define our binary variables for the teachers
X = {}
for t in all_teachers:
    for c in all_courses:
        for b in all_blocks:
            X[t,c,b] = solver.BoolVar('X[%i,%i,%i]' % (t, c, b))

# Define our binary variables for the students
Y = {}
for s in all_students:
    for c in all_courses:
        for b in all_blocks:
            Y[s,c,b] = solver.BoolVar('Y[%i,%i,%i]' % (s, c, b))
            
# Define our Suitability Coefficients for the teachers
Suitability = [[[0 for b in all_blocks] for c in all_courses] for t in all_teachers]
for t in all_teachers:
    for c in all_courses:
        for b in all_blocks:
            Suitability[t][c][b]=TeacherCourseMatrix[t][c]
            +TeacherBlockMatrix[t][b]+CourseBlockMatrix[c][b]

# Define our Preference Coefficients for the students
Preference = [[[0 for b in all_blocks] for c in all_courses] for s in all_students]
for s in all_students:
    for c in all_courses:
        for b in all_blocks:
            Preference[s][c][b]=StudentCourseMatrix[s][c]

# Define our objective function
solver.Maximize(solver.Sum([Suitability[t][c][b] * X[t,c,b] for t in all_teachers 
                            for c in all_courses for b in all_blocks])
               +solver.Sum([Preference[s][c][b] * Y[s,c,b] for s in all_students 
                            for c in all_courses for b in all_blocks]))


# CONSTRAINTS
# Each teacher teaches exactly two courses
for t in all_teachers:
    solver.Add(solver.Sum([X[t,c,b] for c in all_courses for b in all_blocks]) <= 2)

# Each teacher teaches at most once in any slot
for t in all_teachers:
    for b in all_blocks:
        solver.Add(solver.Sum([X[t,c,b] for c in all_courses]) <= 1)

# Each course is covered in one block by one teacher
for c in all_courses:
    solver.Add(solver.Sum([X[t,c,b] for t in all_teachers for b in all_blocks]) == 1)  

# Each block has exactly three courses
for b in all_blocks:
    solver.Add(solver.Sum([X[t,c,b] for t in all_teachers for c in all_courses]) == 3)  

# Each student must take one course each block
for s in all_students:
    for b in all_blocks:
        solver.Add(solver.Sum([Y[s,c,b] for c in all_courses]) == 1)  
    
# No student can take the same course twice
for s in all_students:
    for c in all_courses:
        solver.Add(solver.Sum([Y[s,c,b] for b in all_blocks]) <= 1)  

# No student can take a course in a block when that course is not offered
for c in all_courses:
    for b in all_blocks:
        for s in all_students:
            solver.Add(Y[s,c,b] <= solver.Sum([X[t,c,b] for t in all_teachers]))  

            
# Read in current time. Next, subtract starting time from current time, 
# obtaining the total solving time
current_time = time.time() 
reading_time = current_time - start_time  
# Call the MIP solver
sol = solver.Solve()
# Calculate total solving time
solving_time = time.time() - current_time

# Print summary statistics
print()
print('Optimization Complete with Total Happiness Score of', 
      round(solver.Objective().Value()))
print()
print('Our program needed', round(reading_time,3), 'seconds to read the data and', 
      round(solving_time,3), 'seconds to determine the optimal solution')
print()


#________________________________________________________________________________________
# STEP 4 - OBTAIN AN OUTPUT TIMETABLE
# Format output for teachers
TeacherCourseOutput = TeacherCourseData.copy()

for c in CourseList:
    for t in all_teachers:
        TeacherCourseOutput[c][t]="-"

for t in all_teachers:
    for c in all_courses:
        for b in all_blocks:
            if X[t,c,b].solution_value() > 0:
                TeacherCourseOutput[CourseList[c]][t]=BlockList[b]

# Format output for students            
StudentCourseOutput = StudentCourseData.copy()

for c in CourseList:
    for s in all_students:
        StudentCourseOutput[c][s]="-"

for s in all_students:
    for c in all_courses:
        for b in all_blocks:
            if Y[s,c,b].solution_value() > 0:
                StudentCourseOutput[CourseList[c]][s]=BlockList[b]

# Print the optimal timetable for both teachers and students               
print(TeacherCourseOutput)
print()
print(StudentCourseOutput)

  Student C1 C2 C3 C4 C5 C6 C7 C8 C9 C10 C11 C12
0      S1  Y  Y  Y  -  Y  -  -  -  -   -   -   -
1      S2  -  Y  Y  -  Y  Y  -  -  -   -   -   -
2      S3  -  -  -  Y  -  -  -  -  -   Y   Y   Y
3      S4  Y  Y  -  -  -  -  -  -  Y   -   -   Y
4      S5  -  -  -  -  -  Y  Y  Y  Y   -   -   -
5      S6  -  -  -  Y  -  -  -  Y  -   Y   -   Y
6      S7  Y  -  -  -  -  -  Y  -  -   Y   Y   -
7      S8  -  Y  Y  -  -  Y  Y  -  -   -   -   -

  Teacher C1 C2 C3 C4 C5 C6 C7 C8 C9 C10 C11 C12
0      T1  Y  -  -  -  -  -  Y  -  Y   -   -   -
1      T2  Y  -  -  -  -  Y  Y  -  Y   -   -   -
2      T3  -  Y  -  Y  -  -  -  Y  -   -   -   -
3      T4  Y  Y  -  -  -  -  -  -  -   -   Y   Y
4      T5  -  -  Y  Y  Y  -  -  -  -   -   -   -
5      T6  -  -  Y  -  -  -  -  Y  Y   Y   -   -

  Teacher B1 B2 B3 B4
0      T1  -  -  N  N
1      T2  -  -  -  -
2      T3  -  -  -  N
3      T4  N  -  -  -
4      T5  -  -  -  -
5      T6  N  N  -  -

   Course B1 B2 B3 B4
0      C1  -  -  N  N
1      C2  N  N